In [4]:
import pandas as pd
from collections import defaultdict

In [5]:
df1 = pd.read_csv("100k.txt", header=None)

In [6]:
df1 = df1[0].str.split('\t', expand=True).rename(columns = {0:'SUBJECT',1:'ATTRIBUTE',2:'OBJECT'})

In [7]:
df1['OBJECT'] = df1['OBJECT'].str[:-2] 

In [8]:
groups = df1.groupby('ATTRIBUTE')[['SUBJECT','OBJECT']]

In [9]:
dfs = {}
for attribute in ['follows','friendOf', 'likes', 'hasReview']:
    dfs[attribute] =  groups.get_group(df1[df1['ATTRIBUTE'].str.contains(attribute)]['ATTRIBUTE'].unique()[0])

In [117]:
def hashJoin(dfs):
    keys = list(dfs.keys())
    for i in range(len(dfs)-1):
        h = defaultdict(list)
        # hash phase
        if i == 0:            
            for ind,row in dfs[keys[i]].iterrows():
                h[row['OBJECT']].append(list(row))
        else:
            for ind,row in res_df.iterrows():
                h[row['OBJECT']].append(list(row))
        # join phase
        res_df = pd.DataFrame((list([*(s[:i+1]),r[1],r[1]]) for ind,r in dfs[keys[i+1]].iterrows() for s in h[r['SUBJECT']])).rename(columns = {i+2:'OBJECT'})
    res_df = res_df.drop(len(dfs)-1,axis=1).rename(columns = {'OBJECT':keys[len(dfs)-1]})
    for i in range(len(dfs)-1):
        res_df.rename(columns={i: keys[i]}, inplace=True)
    return res_df

In [ ]:
x = hashJoin(dfs)

In [9]:
x

,follows,friendOf,likes,hasReview
0,wsdbm:User630,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
1,wsdbm:User26,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
2,wsdbm:User197,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
3,wsdbm:User351,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
4,wsdbm:User396,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
...,...,...,...,...
11415456,wsdbm:User186,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248
11415457,wsdbm:User221,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248
11415458,wsdbm:User361,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248
11415459,wsdbm:User762,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248


In [211]:
def sortMergeJoin(dfs):
    keys = list(dfs.keys())
    for i in range(len(dfs)-1):
        if i == 0:
            table_1 = dfs[keys[i]].copy()
        else:
            table_1 = res_df.copy()
        table_2 = dfs[keys[i+1]].copy()
        # Sort step
        table_1.sort_values(by=['OBJECT'], inplace = True)
        table_2.sort_values(by=['SUBJECT'], inplace = True)
        # Merge Step
        tb1_p, tb2_p, rows = 0, 0, []
        while(tb1_p < len(table_1) and tb2_p < len(table_2)):
            vl1 = table_1.iloc[[tb1_p]]['OBJECT'].item()
            vl2 = table_2.iloc[[tb2_p]]['SUBJECT'].item()
            if vl1 == vl2:
                #  FASTER APPROACH BUT USES DATAFRAME FEATURES
                #  subset_1 = table_1[table_1['OBJECT'] == vl1]
                #  subset_2 = table_2[table_2['SUBJECT'] == vl1]
                #  rows += [[*(s[:i+1]),r[1],r[1]] for ind1,s in subset_1.iterrows() for ind2,r in subset_2.iterrows()]
                #  tb1_p += len(subset_1)
                #  tb2_p += len(subset_2)
                rows.append([*table_1.iloc[[tb1_p]].values.tolist()[0][:i+1], *[table_2.iloc[[tb2_p]]['OBJECT'].item()]*2])
                temp_tb1_p = tb1_p + 1
                temp_tb2_p = tb2_p + 1
                while(temp_tb2_p < len(table_2)):
                    vl2_new = table_2.iloc[[temp_tb2_p]]['SUBJECT'].item()
                    if vl1 != vl2_new:
                        break
                    rows.append([*table_1.iloc[[tb1_p]].values.tolist()[0][:i+1], *[table_2.iloc[[temp_tb2_p]]['OBJECT'].item()]*2])
                    temp_tb2_p += 1
                while(temp_tb1_p < len(table_1)):
                    vl1_new = table_1.iloc[[temp_tb1_p]]['OBJECT'].item()
                    if vl1_new != vl2:
                        break
                    rows.append([*table_1.iloc[[temp_tb1_p]].values.tolist()[0][:i+1], *[table_2.iloc[[tb2_p]]['OBJECT'].item()]*2])
                    temp_tb1_p += 1
                tb1_p += 1
                tb2_p += 1
            elif vl1 < vl2:
                tb1_p += 1
            else:
                tb2_p +=1
        res_df = pd.DataFrame(rows).rename(columns = {i+2:'OBJECT'})
    res_df = res_df.drop(len(dfs)-1,axis=1).rename(columns = {'OBJECT':keys[len(dfs)-1]})
    for i in range(len(dfs)-1):
        res_df.rename(columns={i: keys[i]}, inplace=True)
    return res_df       

In [ ]:
y = sortMergeJoin(dfs)

In [213]:
y

,follows,friendOf,likes,hasReview
0,wsdbm:User433,wsdbm:User687,wsdbm:Product0,wsdbm:Review24
1,wsdbm:User433,wsdbm:User687,wsdbm:Product0,wsdbm:Review1074
2,wsdbm:User433,wsdbm:User687,wsdbm:Product0,wsdbm:Review1107
3,wsdbm:User433,wsdbm:User687,wsdbm:Product0,wsdbm:Review1144
4,wsdbm:User433,wsdbm:User687,wsdbm:Product0,wsdbm:Review1158
...,...,...,...,...
11415456,wsdbm:User903,wsdbm:User598,wsdbm:Product9,wsdbm:Review1498
11415457,wsdbm:User836,wsdbm:User598,wsdbm:Product9,wsdbm:Review1498
11415458,wsdbm:User980,wsdbm:User598,wsdbm:Product9,wsdbm:Review1498
11415459,wsdbm:User993,wsdbm:User598,wsdbm:Product9,wsdbm:Review1498


In [121]:
# Compared to built in pandas merge function
dfs['follows'].merge(dfs['friendOf'], left_on = 'OBJECT', right_on = 'SUBJECT').rename(columns = {'OBJECT_y':'OBJECT'})\
              .merge(dfs['likes'], left_on = 'OBJECT', right_on = 'SUBJECT').rename(columns = {'OBJECT_y':'OBJECT'})\
              .merge(dfs['hasReview'], left_on = 'OBJECT', right_on = 'SUBJECT')

C:\Users\sajus\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'OBJECT_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
C:\Users\sajus\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'SUBJECT_x', 'OBJECT_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,SUBJECT_x,OBJECT_x,SUBJECT_y,OBJECT_x,SUBJECT_x,OBJECT_x,SUBJECT_y,OBJECT_y
0,wsdbm:User579,wsdbm:User10,wsdbm:User10,wsdbm:User614,wsdbm:User614,wsdbm:Product0,wsdbm:Product0,wsdbm:Review24
1,wsdbm:User579,wsdbm:User10,wsdbm:User10,wsdbm:User614,wsdbm:User614,wsdbm:Product0,wsdbm:Product0,wsdbm:Review981
2,wsdbm:User579,wsdbm:User10,wsdbm:User10,wsdbm:User614,wsdbm:User614,wsdbm:Product0,wsdbm:Product0,wsdbm:Review247
3,wsdbm:User579,wsdbm:User10,wsdbm:User10,wsdbm:User614,wsdbm:User614,wsdbm:Product0,wsdbm:Product0,wsdbm:Review253
4,wsdbm:User579,wsdbm:User10,wsdbm:User10,wsdbm:User614,wsdbm:User614,wsdbm:Product0,wsdbm:Product0,wsdbm:Review282
...,...,...,...,...,...,...,...,...
11415456,wsdbm:User152,wsdbm:User966,wsdbm:User966,wsdbm:User617,wsdbm:User617,wsdbm:Product141,wsdbm:Product141,wsdbm:Review1363
11415457,wsdbm:User152,wsdbm:User966,wsdbm:User966,wsdbm:User617,wsdbm:User617,wsdbm:Product141,wsdbm:Product141,wsdbm:Review1367
11415458,wsdbm:User152,wsdbm:User966,wsdbm:User966,wsdbm:User617,wsdbm:User617,wsdbm:Product141,wsdbm:Product141,wsdbm:Review1401
11415459,wsdbm:User152,wsdbm:User966,wsdbm:User966,wsdbm:User617,wsdbm:User617,wsdbm:Product141,wsdbm:Product141,wsdbm:Review1489
